# Confronto tra Insertion Sort e Counting Sort
## Matteo Lotti
### Marzo-Aprile 2023

In questo notebook ci poniamo l'obiettivo di impostare e effettuare un confronto tra due algoritmi di ordinamento. In particolare, i due algoritmi in questione saranno:

- __Insertion Sort__

- __Counting Sort__

Il notebook sarà sviluppato in questo modo:

1. Nelle celle successive saranno presenti le implementazioni dei due algoritmi seguiti rispettivamente da una breve spiegazione messa in relazione con le evidenze teoriche conosciute.

2. Successivamente sarà presente una cella nella quale saranno implementate le funzioni di test, seguita da un'opportuna spiegazione del codice e dei test che saranno eseguiti

3. Saranno effettuati e descritti i dovuti test e i rispettivi risultati

4. Nella chiusura tireremo le somme dei risultati ottenuti dagli esperimenti svolti

## Implementazioni degli algoritmi
### Insertion Sort

In [184]:
def insertion_sort(arr):
    """Insertion sort algorithm.

    Args:
        arr (list): Array to be sorted.

    Returns:
        list: Sorted array.

    Time complexity: O(n^2)
    """
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and key < arr[j]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
    return arr

Nella cella sovrastante possiamo vedere l'implementazione dell'algoritmo di ordinamento __Insertion Sort__. L'algoritmo si basa su un ciclo _for_ iniziale nel quale si inizializza la variabile _key_ con il valore all'indice _i_ dell'array da ordinare. All'interno del _for_ un ciclo _while_ ci permette di iterare sugli elementi precedenti a _key_ in modo tale da posizionare il valore di _key_ nella posizione corretta. Al termine di ogni esecuzione del ciclo _for_, i primi _i_ elementi saranno ordinati e quando termina l'algoritmo tutto l'array sarà ordinato correttamente.

Il tempo di esecuzione dell'algoritmo è pari a _O(n^2)_ in quanto ogni elemento viene confrontato con tutti gli elementi precedenti a lui.

Nel caso peggiore, l'array è ordinato in modo inverso, quindi l'algoritmo dovrà effettuare _n-1_ confronti per il primo elemento, _n-2_ per il secondo, _n-3_ per il terzo e così via. Quindi il numero totale di confronti sarà pari a: $$\sum_{i=1}^{n-1} i = \frac{(n-1)(n)}{2} = \frac{n^2-n}{2}$$
e il tempo di esecuzione sarà pari a: $$\frac{n^2-n}{2} \cdot c$$ dove _c_ è il tempo di esecuzione di un singolo confronto. Percò il tempo di esecuzione è sempre pari a _Θ(n^2)_.

Nel caso migliore, l'array è già ordinato, quindi l'algoritmo non dovrà effettuare alcun confronto. Quindi il tempo di esecuzione sarà pari a: $$n \cdot c$$ dove _c_ è il tempo di esecuzione di un singolo confronto.

### Counting Sort

In [185]:
def counting_sort(arr):
    """Counting sort algorithm.

    Args:
        arr (list): Array to be sorted.

    Returns:
        list: Sorted array.

    Time complexity: O(n+k)

    k is the value of the maximum element in the array.
    """
    if not arr:
        return arr
    max_element = max(arr) #k
    count_array = [0] * (max_element + 1) #primo for

    for element in arr:
        count_array[element] += 1

    for i in range(1, len(count_array)):
        count_array[i] += count_array[i-1]

    sorted_array = [0] * len(arr)
    for element in reversed(arr):
        sorted_array[count_array[element]-1] = element #-1 perché indicizzazione parte da 0
        count_array[element] -= 1

    return sorted_array


Nella cella sovrastante troviamo l'algoritmo di ordinamento __Counting Sort__. Sottlineamo il fatto che questo algoritmi è utilizzabile solo per liste di numeri interi. Non è un algoritmo che opera per confronti, ma si basa sul determinare la posizione di ogni elemento nell'array ordinato in base a quanti sono gli elementi minori o uguali ad esso. In particolare, è necessario un array di appoggio di dimensione _max_element_, dove _max_element_ non è altro che il valore massimo presente nell'array di partenza. Successivamente, dopo aver inzializzato quest'array con tutti 0, incrementiamo di 1 l'elemento di indice _i_ per ogni elemento uguale a _i_ nell'array iniziale; poi scorriamo l'array di appoggio sommando ad ogni elemento tutti gli elementi ad esso precedenti. Quello che otteniamo dopo queste due operazioni non è altro che un array di appoggio in cui l'elemento di indice _x_ ha come valore il numero di elementi minori o uguali a _x_ nell'array iniziale. L'ultimo passaggio è quello di copiare su un nuovo array gli elementi ordinati, considerando che nell'array di appoggio l'elemento con indice _x_ risulta essere la posizione di _x_ nell'array ordinato. Decrementando poi progressivamente i valori dell'array di appoggio si riescono a gestire anche ventuali copie multiple di uno stesso valore.

L'algoritmo scorre per due volte l'array iniziale e per due volte l'array di appoggio. Per l'array iniziale il costo sarà _Θ(n)_ dove _n_ è la dimensione dell'array, mentre per l'array di appoggio il costo sarà _Θ(k)_ dove _k_ non èaltro che il massimo valore presente nell'array iniziale Quindi il tempo di esecuzione è sempre pari a _Θ(n+k)_, che diventa _Θ(n)_ nel caso in cui _k=O(n)_.

## Implementazione e spiegazione delle funzioni di test

In [186]:
import pandas as pd
import plotly.express as px
from enum import Enum
from dataclasses import dataclass, field
from timeit import default_timer as timer
import numpy as np


class InputType(Enum):
    """Select the type of input."""

    random = 1
    """Random input."""
    sorted = 2
    """Sorted input."""
    reversed = 3
    """Reversed input."""

class SelectTestType(Enum):
    """Select the type of sorting test."""

    insertion_sort = 1
    """Insertion sort."""
    counting_sort = 2
    """Counting sort."""

@dataclass
class InputConfig:
    """Input configuration."""

    num_samples: int = 1000
    """Number of samples."""
    sample_range: tuple[int, int] = (0, 5000)
    """Range of the samples."""
    input_type: InputType = InputType.random
    """Type of input."""


@dataclass
class InputGenerator:
    """Input generator."""

    input_config: InputConfig = InputConfig()
    """Input configuration."""

    data: list[int] = field(init=False, default_factory=list)
    """Data to be used for the tests."""

    def __post_init__(self):
        """Initialize the data."""
        self.data = self._generate()

    def _generate(self) -> list[int]:
        """Generate the data."""
        match self.input_config.input_type:
            case InputType.random:
                data = np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                )
            case InputType.sorted:
                data = np.sort(np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                ))
            case InputType.reversed:
                data = np.sort(np.random.randint(
                    self.input_config.sample_range[0],
                    self.input_config.sample_range[1],
                    self.input_config.num_samples,
                ))
                data = data[::-1]
            case _:
                raise ValueError("Invalid input type.")
        return data.tolist()

def insertion_test(input_data: list[int]) -> float:
    """Test the insertion sort algorithm."""
    start = timer()
    insertion_sort(input_data)
    end = timer()
    return end - start

def counting_test(input_data: list[int]) -> float:
    """Test the counting sort algorithm."""
    start = timer()
    counting_sort(input_data)
    end = timer()
    return end - start

def test(test_type: SelectTestType, input_type: InputType, num_samples: int, sample_range: tuple[int, int], step: int) -> pd.DataFrame:
    """Test the sorting algorithms.

    Args:
        test_type (SelectTestType): Type of sorting algorithm to test.
        input_type (InputType): Type of input.
        num_samples (int): Number of elements to sort.
        sample_range (tuple[int, int]): Range of the elements.
        step (int): Step to increase the number of elements.

        Returns:
            pd.DataFrame: Dataframe containing the results."""

    match test_type:
        case SelectTestType.insertion_sort:
            test_func = insertion_test
        case SelectTestType.counting_sort:
            test_func = counting_test
        case _:
            raise ValueError("Invalid queue type.")
    match input_type:
        case InputType.random:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.random)
        case InputType.sorted:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.sorted)
        case InputType.reversed:
            input_config = InputConfig(num_samples=num_samples, sample_range=sample_range, input_type=InputType.reversed)
        case _:
            raise ValueError("Invalid input type.")

    it = []
    for i in np.arange(0, num_samples, step):
        input_config.num_samples = i
        input_gen = InputGenerator(input_config=input_config)
        it.append(test_func(input_gen.data))

    times_df = pd.DataFrame(data={
        "num_samples": np.arange(0, num_samples, step),
        "time": it,
    })
    times_df["test_type"] = test_type.name
    times_df["input_type"] = input_type.name
    return times_df




Nella cella sovrastante sono state definite le funzioni e le classi necessarie per eseguire i test. In particolare, sono state definite le seguenti classi:
* _SelectTestType_: enum che contiene i tipi di test disponibili (insertion sort e counting sort)
* _InputType_: enum che contiene i tipi di input disponibili
* _InputConfig_: contiene i parametri necessari per generare l'input (numero di elementi, range degli elementi, tipo di input)
* _InputGenerator_: genera l'input in base ai parametri specificati

Inoltre, sono state definite le seguenti funzioni:
* _insertion_test_: esegue il test dell'algoritmo di insertion sort
* _counting_test_: esegue il test dell'algoritmo di counting sort
* _test_: esegue il test di uno dei due algoritmi di ordinamento, in base al tipo di test e di input scelti dall'utente. In particolare, questa funzione:
    * in base al tipo di test scelto, assegna alla variabile _test_func_ la funzione di test corrispondente
    * in base al tipo di input scelto, crea un oggetto _InputConfig_ con i parametri corrispondenti
    * i test vengono eseguiti su un numero di elementi che va da 0 a _num_samples_ con passo _step_
    * crea un oggetto _InputGenerator_ con l'oggetto _InputConfig_ creato precedentemente
    * esegue il test _test_func_ sull'input generato
    * crea un dataframe contenente i risultati del test e lo restituisce

    Il dataframe contiene le seguenti colonne:
    * _num_samples_: numero di elementi ordinati
    * _time_: tempo impiegato per ordinare gli elementi
    * _test_type_: tipo di test eseguito (insertion sort o counting sort)
    * _input_type_: tipo di input utilizzato (random, ordinato o inversamente ordinato)

In [187]:
def plot_results(num_samples: int, step: int, sample_range: tuple[int, int]):
    """Plot the results."""

    """Insertion sort tests"""

    df_ins_1 = test(SelectTestType.insertion_sort, InputType.random, num_samples, sample_range, step)
    df_ins_2 = test(SelectTestType.insertion_sort, InputType.sorted, num_samples, sample_range, step)
    df_ins_3 = test(SelectTestType.insertion_sort, InputType.reversed, num_samples, sample_range, step)

    """Counting sort tests"""

    df_cnt_1 = test(SelectTestType.counting_sort, InputType.random, num_samples, sample_range, step)
    df_cnt_2 = test(SelectTestType.counting_sort, InputType.sorted, num_samples, sample_range, step)
    df_cnt_3 = test(SelectTestType.counting_sort, InputType.reversed, num_samples, sample_range, step)

    df = pd.concat([df_ins_1, df_ins_2, df_ins_3, df_cnt_1, df_cnt_2, df_cnt_3], ignore_index=True)

    return df



Nella cella sovrastante è stata definita la funzione _plot_results_ che esegue tutti i test prendendo in input il numero di elementi dell'array, il passo con il quale devono essere eseguiti i test e il range di generazione dei numeri; restituisce un dataframe Pandas contenente i risultati dei test.
Nelle celle successive in cui viene eseguito codice, si mostrano i grafici voluti in base ai parametri scelti dall'utente.
In particolare, tramite il framework _plotly_ è possibile creare grafici interattivi che consentono di visualizzare i risultati dei test in maniera più chiara e comprensibile.

## Esecuzione dei test

In questa sezione vengono eseguiti i test di ordinamento e vengono mostrati i grafici relativi ai risultati ottenuti. Nello specifico sono stati eseguiti i seguenti test:

* Array di dimensione 100 con passo pari a 10 e range di generazione dei numeri da 0 a 100 e da 0 a 1000
* Array di dimensione 1000 con passo pari a 10 e range di generazione dei numeri da 0 a 10000, da 0 a 100000, da 0 a 1000000
* Array di dimensione 10000 con passo pari a 100 e range di generazione dei numeri da 0 a 100000 e da 0 a 1000000

In tutti i casi, i test sono stati eseguiti su array di numeri casuali, ordinati e inversamente ordinati.
I grafici saranno interattivi in modo da poter visualizzare i risultati dei test di insetrtion sort e counting sort anche in maniera separata.

La scelta delle dimensioni degli input è stata fatta in modo da poter avere un panoramica completa sui risultati dei test, sia per array di piccole dimensioni, sia per array di grandi dimensioni; analoga scelta è stata fatta per il range di generazione dei numeri, che risulterà di particolare importanza nel caso dell'algoritmo _counting sort_.

Sottolineiamo inoltre che al momento della visulizzazione dei grafici è stato scelto di escludere il test su array di dimensione 0 in quanto non ha particolare senso eseguirlo.


### 1. Array di dimensione 100

Analizziamo i test eseguiti su array di dimensione 100 con passo pari a 10 e range di generazione dei numeri da 0 a 100 e da 0 a 1000.
I grafici mostrano i risultati dei test di insertion sort e counting sort in base al tipo di input utilizzato. Per questo caso è stato scelto di utilizzare grafici a dispersione, in modo da poter visualizzare in maniera più chiara i risultati dei test.

#### Array di dimensione 100 con range da 0 a 100

In [201]:
df = plot_results(100, 10, (0, 100))

In [202]:
figure = px.scatter(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})

figure.show()

    num_samples          time       test_type input_type
0             0  2.100016e-06  insertion_sort     random
1            10  5.799986e-06  insertion_sort     random
2            20  1.450005e-05  insertion_sort     random
3            30  2.700003e-05  insertion_sort     random
4            40  5.899998e-05  insertion_sort     random
5            50  1.104000e-04  insertion_sort     random
6            60  1.801000e-04  insertion_sort     random
7            70  2.425000e-04  insertion_sort     random
8            80  2.727000e-04  insertion_sort     random
9            90  3.086000e-04  insertion_sort     random
10            0  2.999965e-06  insertion_sort     sorted
11           10  3.799971e-06  insertion_sort     sorted
12           20  6.099988e-06  insertion_sort     sorted
13           30  5.600043e-06  insertion_sort     sorted
14           40  5.599984e-06  insertion_sort     sorted
15           50  6.799994e-06  insertion_sort     sorted
16           60  9.500014e-06  

Visualizziamo il caso con passo pari a 10 e range da 0 a 100. Partendo dall' algoritmo di _insertion sort_, notiamo che nel caso di un input randomico o inversamente ordinato, il tempo impiegato per ordinare l'array cresce in modo quadratico e lo fa in modo particolare nel secondo caso. Nel caso di un input ordinato, invece, il tempo impiegato per ordinare l'array è lineare, in quanto l'array è già ordinato e non è necessario eseguire alcuna operazione di ordinamento, ma solo il ciclo di "controllo" su tutti i valori. Visualizzeremo questo aspetto in maniera più chiara nel grafico successivo, dove isoleremo il caso di un input ordinato. Nel caso del _counting sort_, invece, notiamo che il tempo impiegato per ordinare l'array è lineare (visibile deselezionando la dicituara _insertion_sort_ sotto il menù _Algoritmo di ordinamento_), in quanto l'array viene ordinato in tempo _O(n+k)_, senza alcuna operazione di confronto tra elementi. Notiamo che in questo caso il costo computazionale del _counting sort_ è particolarmente basso, in quanto legato al range di generazione dei numeri, che è molto piccolo (da 0 a 100). Questo aspetto sarà più evidente nel caso di array di grandi dimensioni ma anche nei casi con range di generazione dei numeri più grandi.

In [190]:
figure = px.line(df.query("num_samples > 0 and input_type == 'sorted'"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})

figure.show()

Come detto in precedenza, nel caso di un input ordinato, l'array è già ordinato e non è necessario eseguire alcuna operazione di ordinamento, ma solo il ciclo di "controllo" su tutti i valori. Questo aspetto è evidente nel grafico sovrastante, dove è possibile notare che l'andamento del tempo impiegato per ordinare l'array è lineare. Deselezionando la dicitura _counting_sort_ sotto il menù _Algoritmo di ordinamento_ questo aspetto è ancora più evidente (considerando il fatto che il tempo impiegato è molto basso e quindi è possibile che ci sia qualche piccola variazione nell'andamento).

#### Array di dimensione 100 con range da 0 a 1000

In [191]:
df = plot_results(100, 10, (0, 1000))

In [192]:
figure = px.scatter(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})

figure.show()

Se il range di generazione dei numeri è fino a 1000, il tempo impiegato dal _counting sort_ per ordinare l'array è molto più elevato, in quanto il costo computazionale del _counting sort_ è legato al range di generazione dei numeri; più è elevato il valore massimo dell'array, più il costo computazionale sarà elevato.

Nelle sezioni successive i grafici saranno continui, in modo tale che gli andamenti dei test siano più chiari e visibili.

### 2. Array di dimensione 1000

In questo caso, analizziamo i test eseguiti su array di dimensione 1000 con passo pari a 10 e range di generazione dei numeri da 0 a 10000, da 0 a 100000 e da 0 a 1000000. I grafici mostrano i risultati dei test di insertion sort e counting sort in base al tipo di input utilizzato.

#### Array di dimensione 1000 con range da 0 a 1000

In [193]:
df = plot_results(1000, 10, (0, 10000))

In [194]:
figure = px.line(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})
figure.show()

Notiamo che in questo caso è molto più evidente il fatto che la complessità computazionale dell' _insertion sort_ è _O(n^2)_, in quanto il tempo impiegato per ordinare l'array cresce in modo quadratico. Nel caso del _counting sort_, invece, notiamo che il tempo impiegato per ordinare l'array è lineare (deselezionando _insetion_sort_ nel menù a destra), in quanto l'array viene ordinato in tempo _O(n+k)_, senza alcuna operazione di confronto tra elementi. Notiamo che in questo caso il costo computazionale del _counting sort_ è particolarmente basso, in quanto legato al range di generazione dei numeri, che è ancora piuttosto piccolo rispetto alla dimensione dell'array (da 0 a 10000). L'aumento del costo computazionale sarà ancora più visibile nel caso di array di grandi dimensioni ma anche nei casi con range di generazione dei numeri più grandi.

#### Array di dimensione 1000 con range da 0 a 100000

In [195]:
df = plot_results(1000, 10, (0, 100000))

In [196]:
figure = px.line(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})
figure.show()

Aumentando il range di generazione dei numeri, il tempo impiegato dal _counting sort_ per ordinare l'array è molto più elevato, in quanto il costo computazionale del _counting sort_ è legato al range di generazione dei numeri; più è elevato il valore massimo dell'array, più il costo computazionale sarà elevato. Notiamo che comunque il tempo impiegato dal _counting sort_ è sempre lineare, aldilà di qualche piccola variazione nell'andamento nell'ordine dei millisecondi, dovuta probabilmente anche al fatto che il tempo impiegato per eseguire il test è molto basso.

#### Array di dimensione 1000 con range da 0 a 1000000

In [199]:
df = plot_results(1000, 10, (0, 1000000))

In [200]:
figure = px.line(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})
figure.show()

In quest'ultimo test aumentiamo in modo considerevole il range rispetto alla dimensione dell'array e il risultato è evidente; il costo computazionale del _counting sort_ è molto elevato, in quanto il tempo impiegato per ordinare l'array è molto più elevato rispetto ai casi precedenti. Notiamo che il tempo impiegato dal _counting sort_ è sempre lineare, ma di gran lunga superiore rispetto al costo computazionale dell' _insertion sort_ e rispetto ai casi precedenti.

### 3. Array di dimensione 10000

In questa sezione di test, analizziamo i test eseguiti su array di dimensione 10000 con passo pari a 100 e range di generazione dei numeri da 0 a 10000 e da 0 a 100000. I grafici mostrano i risultati dei test di insertion sort e counting sort in base al tipo di input utilizzato. Aumentiamo quindi in modo considerevole sia la dimensione dell'array che il range di generazione dei numeri.

#### Array di dimensione 10000 con range da 0 a 100000

In [204]:
df = plot_results(10000, 100, (0, 100000))

In [205]:
figure = px.line(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})
figure.show()

Notiamo che in ogni caso mantenendo il range di generazione dei numeri minore o uguale alla dimensione dell'array, il costo compurazionale del _counting sort_ si mantiene piuttosto basso o comunque in ogni caso molto inferiore rispetto al costo di _insertion sort_, che è sempre _O(n^2)_. Ovviamente l'unica differenza sta nel caso di input ordinato, nel quale anche il costo di _insertion sort_ è lineare e quindi molto inferiore rispetto al caso di input random o inversamente ordinato.

#### Array di dimensione 10000 con range da 0 a 1000000

In [206]:
df = plot_results(10000, 100, (0, 1000000))

In [207]:
figure = px.line(df.query("num_samples > 0"),
                 x="num_samples",
                 y="time",
                 color="test_type",
                 facet_col="input_type",
                 title="Sorting algorithms",
                 labels={"num_samples": "Dimensione array", "time": "Time (s)", "test_type": "Algoritmo di ordinamento", "input_type": "Tipo di input"})
figure.show()

Notiamo che anche decuplicando il range di generazione dei numeri, il costo computazionale del _counting sort_ aumenta, ma in modo molto meno evidente rispetto al caso precedente, soprattutto in relazione al costo di _insertion sort_, che per un numero di elementi elevato aumenta considerevolmente.
Di conseguenza per array di grandi dimensioni, dovremmo aumentare il range di generazione dei numeri in modo considerevole per ottenere un costo computazionale del _counting sort_ simile o superiore a quello di _insertion sort_.

## Analisi dei risultati dei test e conclusioni

I nostri test hanno dimostrato che il costo computazionale del _counting sort_ è lineare (in particolare _O(n+k)_) in quanto il tempo impiegato per ordinare l'array è sempre proporzionale alla dimensione dell'array e al range di generazione dei numeri. Inoltre, il costo computazionale del _counting sort_ è generalmente molto inferiore rispetto al costo computazionale dell' _insertion sort_, che è sempre _O(n^2)_. Ovviamente le uniche "anomalie" sono:
   * caso di input ordinato, che è il caso migliore per l'esecuzione dell'algortimo di _insertion sort_, in cui il costo computazionale è lineare e quindi molto inferiore rispetto al caso di input random o inversamente ordinato.
   * caso di input con range di generazione dei numeri molto maggiore della dimensione dell'array, in cui il costo computazionale del _counting sort_ è molto elevato, potenzialmente anche di più dell'algoritmo di _insertion sort_.

In conclusione, possiamo dire che il _counting sort_ è un algoritmo molto efficiente, in quanto il costo computazionale è lineare e quindi generalmente molto inferiore rispetto al caso di _insertion sort_, che è sempre _O(n^2)_. Ricordiamo però che l'utilizzo del _counting sort_ è limitato al fatto che l'array di input deve contenere solo numeri interi positivi, in quanto il suo funzionamento è basato sul conteggio degli elementi presenti nell'array. Ovviamente, l'algoritmo _counting sort_ non è consigliato nei due casi precedenti, ma sicuramente è molto utile per ordinare array di numeri interi, in particolare per array di grandi dimensioni.